In [ ]:
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt
import scipy.stats as stats


In [ ]:
_BASE_URL = "https://raw.githubusercontent.com/rmcelreath/rethinking/Experimental/data"
HOWELL_DATASET_PATH = f"{_BASE_URL}/Howell1.csv"
d = pd.read_csv(HOWELL_DATASET_PATH, sep=';', header=0)
d = d[d.age >= 18]  # just adults
d.head()

In [ ]:
model_ppc=CmdStanModel(stan_file='height_1_ppc.stan')

R = 1000
sim=model_ppc.sample(iter_sampling=R,
                     iter_warmup=0,
                     chains=1,
                     fixed_param=True,
                     seed=29042020,refresh=R)

In [ ]:
# Prior predictive checks
model_ppc = CmdStanModel(stan_file='height_1_ppc.stan')

R = 1000
sim = model_ppc.sample(iter_sampling=R,
                       iter_warmup=0,
                       chains=1,
                       fixed_param=True,
                       seed=29042020, refresh=R)

# Plot histograms of mu, sigma, and simulated height
plt.hist(sim['mu'], bins=50, density=True, color='blue', alpha=0.7, label='mu')
plt.hist(sim['sigma'], bins=50, density=True, color='red', alpha=0.7, label='sigma')
plt.hist(sim['height_sim'], bins=50, density=True, color='green', alpha=0.7, label='simulated height')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Histograms of mu, sigma, and simulated height')
plt.legend()
plt.show()

# Plot a joint distribution of mu and sigma
plt.scatter(sim['mu'], sim['sigma'], alpha=0.5)
plt.xlabel('mu')
plt.ylabel('sigma')
plt.title('Joint Distribution of mu and sigma')
plt.show()

In [ ]:
# Model fit and evaluation
model_1_fit = CmdStanModel(stan_file='height_1_fit.stan')
fit = model_1_fit.sample(data=dict(N=len(d),
                                   heights=d.height.values),
                         seed=28052020)

# Plot a joint distribution of fitted mu and sigma
az.plot_pair(az.from_cmdstanpy(fit), var_names=['alpha', 'sigma'])
plt.show()

# Plot histograms of data and simulated heights
plt.hist(d.height, bins=30, density=True, color='blue', alpha=0.7, label='Data')
plt.hist(fit['height_sim'], bins=30, density=True, color='red', alpha=0.7, label='Simulated heights')
plt.xlabel('Height')
plt.ylabel('Density')
plt.title('Histograms of Data and Simulated Heights')
plt.legend()
plt.show()

In [ ]:
# Create column c_weight in the dataframe containing weights subtracted by their mean
d['c_weight'] = d['weight'] - d['weight'].mean()

# Define data_sim dictionary containing data for simulation
data_sim = {'N': 50, 'weight': np.linspace(d.c_weight.min(), d.c_weight.max())}

In [ ]:

# Load the Stan model for prior predictive checks
model_ppc = CmdStanModel(stan_file='height_2a_ppc.stan')

# Define the number of iterations
R = 1000

# Sample from the model
sim = model_ppc.sample(data=data_sim,
                       iter_sampling=R,
                       iter_warmup=0,
                       chains=1,
                       refresh=R,
                       fixed_param=True,
                       seed=29042020)

# Plot lines for each sampled slope beta and intercept alpha
plt.figure(figsize=(10, 6))
for i in range(len(sim['alpha'])):
    plt.plot(data_sim['weight'], sim['alpha'][i] + sim['beta'][i] * data_sim['weight'], color='gray', alpha=0.1)

# Verify if possible predicted heights are consistent with minimum (0) and maximum heights observed in nature
# Add horizontal lines for minimum and maximum heights
plt.axhline(y=0, color='r', linestyle='--', label='Minimum Height (0)')
plt.axhline(y=d['height'].max(), color='r', linestyle='--', label='Maximum Height')
plt.xlabel('Weight')
plt.ylabel('Height')
plt.title('Prior Predictive Checks')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Load the Stan model with modified prior for beta
model_ppc = CmdStanModel(stan_file='height_2b_ppc.stan')

# Sample from the model
sim = model_ppc.sample(data=data_sim,
                       iter_sampling=R,
                       iter_warmup=0,
                       chains=1,
                       refresh=R,
                       fixed_param=True,
                       seed=29042020)

# Plot lines for each sampled slope beta and intercept alpha
plt.figure(figsize=(10, 6))
for i in range(len(sim['alpha'])):
    plt.plot(data_sim['weight'], sim['alpha'][i] + sim['beta'][i] * data_sim['weight'], color='gray', alpha=0.1)

# Verify if possible predicted heights are consistent with minimum (0) and maximum heights observed in nature
# Add horizontal lines for minimum and maximum heights
plt.axhline(y=0, color='r', linestyle='--', label='Minimum Height (0)')
plt.axhline(y=d['height'].max(), color='r', linestyle='--', label='Maximum Height')
plt.xlabel('Weight')
plt.ylabel('Height')
plt.title('Modified Prior Predictive Checks for Beta')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Task 6: Fitting and evaluating the model

# Load the Stan model for fitting
model_2_fit = CmdStanModel(stan_file='height_2_fit.stan')

# Create data_fit dictionary containing data from N first rows of dataframe
data_fit = {'N': len(d), 'centered_weight': d.c_weight.values, 'heights': d.height.values}

# Sample from the model
fit = model_2_fit.sample(data=data_fit, seed=28052020)

# Plot lines for each sampled slope beta and intercept alpha
plt.figure(figsize=(10, 6))
for i in range(len(fit['alpha'])):
    plt.plot(d.c_weight, fit['alpha'][i] + fit['beta'][i] * d.c_weight, color='gray', alpha=0.1)

# Verify how uncertainty changes with increasing of sample (N)
# Add error bars for one standard deviation
plt.errorbar(d.c_weight, fit['alpha'].mean() + fit['beta'].mean() * d.c_weight, yerr=fit['sigma'].std(),
             fmt='o', color='blue', label='Mean Simulated Height with Std. Dev.')
plt.xlabel('Centered Weight')
plt.ylabel('Height')
plt.title('Fitted Model with Uncertainty')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Task 7: Extending the model

# Center the weight data (subtract mean value of weight from all values)
d['centered_weight'] = d['weight'] - d['weight'].mean()

# Test how the model works for centered data
# The interpretation of beta in such case is the effect of a one-unit change in centered weight on height

# Modify your model with a second power of weight
# Select prior for its coefficient using prior predictive checks
# After fitting, check if the model is still good

# Define data_fit dictionary containing data from N first rows of dataframe
data_fit = {'N': len(d), 'heights': d.height.values, 'centered_weight': d.centered_weight.values}

# Load the Stan model for fitting the extended model
model_extended = CmdStanModel(stan_file='height_extended.stan')

# Sample from the extended model
fit_extended = model_extended.sample(data=data_fit, seed=28052020)

# Plot lines for each sampled slope beta_1 and beta_2
plt.figure(figsize=(10, 6))
for i in range(len(fit_extended['alpha'])):
    plt.plot(d.centered_weight, fit_extended['alpha'][i] + fit_extended['beta_1'][i] * d.centered_weight +
             fit_extended['beta_2'][i] * d.centered_weight**2, color='gray', alpha=0.1)

# Verify how uncertainty changes with increasing of sample (N)
# Add error bars for one standard deviation
plt.errorbar(d.centered_weight, fit_extended['alpha'].mean() + fit_extended['beta_1'].mean() * d.centered_weight +
              fit_extended['beta_2'].mean() * d.centered_weight**2, yerr=fit_extended['sigma'].std(),
              fmt='o', color='blue', label='Mean Simulated Height with Std. Dev.')
plt.xlabel('Centered Weight')
plt.ylabel('Height')
plt.title('Fitted Model with Uncertainty (Extended)')
plt.legend()
plt.grid(True)
plt.show()
